# Разметка корпуса и работа с ним

На этом семинаре у нас две задачи:

1. Научиться делать морфологическую и синтаксическую разметку корпуса.
2. Научиться работать с ним: извлекать и обобщать информацию.

[Ссылка](https://colab.research.google.com/drive/1xqza3cFYlSGSBCJK4TujgFwwNGdlZEuq?usp=sharing) на тетрадку в Google Colab.

In [1]:
%%capture
!pip install stanza
!pip install pandas

In [2]:
#Выполнение запросов в интернет — используем, чтобы скачать файл
import requests

#Работа с таблицами — понадобится, чтобы хранить наш корпус
import pandas as pd

#Выполнение быстрых математических вычислений — пригодится, потому что иначе
#корпус с большим количеством текстов может долго обрабатываться
import numpy as np

#Разметка морфологии и синтаксиса
import stanza

## Разметка

Будем использовать Stanza — см. [предыдущий конспект](https://github.com/alekseyst/text_analysis_2028/blob/main/Practical_3/Practical_3_Annotation.ipynb).

In [3]:
#Скачиваем обученную для русского языка модель
stanza.download("ru")

#Загружаем модель
nlp = stanza.Pipeline(lang="ru", processors="tokenize, pos, lemma, depparse")

2025-02-15 12:06:10 INFO: Downloaded file to /home/aleksey/stanza_resources/resources.json
2025-02-15 12:06:10 INFO: Downloading default packages for language: ru (Russian) ...
2025-02-15 12:06:11 INFO: File exists: /home/aleksey/stanza_resources/ru/default.zip
2025-02-15 12:06:15 INFO: Finished downloading models and saved to /home/aleksey/stanza_resources
2025-02-15 12:06:15 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-02-15 12:06:15 INFO: Downloaded file to /home/aleksey/stanza_resources/resources.json
2025-02-15 12:06:16 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |

2025-02-15 12:06:16 INFO: Using device: cpu
2025-02-15 12:06:16 INFO: Loading: tokenize
2025-02-15 12:06:17 INFO: Loading: pos
2025-02-15 12:06:18 INFO: Loading: lemma
2025-02-15 12:06:20 INFO: Loading: depparse
2025-02-15 12:06:20 INFO: Done loading processors!


##Разметка длинного текста

Как нам хранить информацию о длинном тексте? Наверное, будет не удобно размечать его каждый раз, когда он нам потребуется. Да и для поиска объект, который хранит Stanza, не самый удобный.

Возможный вариант — преобразовать размеченный текст в таблицу, где в каждой строчке содержится информация об одном слове. Таким образом можно будет сохранить текст в файл — и в следующий раз не парсить текст заново, а только обратиться к уже размеченному файлу. Плюс для поиска и анализа даных в таблице есть удобный инструментарий.

Для работы с таблицами будем использовать библиотеку Pandas.

In [4]:
#Скачиваем текстовый файл с книгой

response = requests.get('https://raw.githubusercontent.com/alekseyst/text_analysis_2024/main/Practical_4/alisa.txt')
aliсe_text = response.text

In [5]:
#Размечаем книгу — это займёт какое-то время.
#Может оказаться быстрее разбить текст на части, но это не обязательно
#Может оказаться быстрее выполнить токенизацию другим инструментом
alice_annotated = nlp(aliсe_text)

In [9]:
#Пройдёмся по каждому слову в книге и добавим в табличку

#Создаём заголовок
cols = ['index', 'word', 'lemma', 'pos', 'feats',
        'synt', 'head']

#Создаём вспомогательный список
words = []

#Добавляем разметку во вспомогательный список
for word_index, word in enumerate(paragraph_nlp.iter_words()):
    words.append([word_index, word.text, word.lemma, word.upos, word.feats,
              word.deprel, word.head])

#Создаём таблицу
aliсe_by_word = pd.DataFrame(words, columns = cols)

In [10]:
aliсe_by_word.head(15)

,index,word,lemma,pos,feats,synt,head
0,0,Ах,ах,INTJ,None,discourse,5
1,1,",",",",PUNCT,None,punct,1
2,2,дело,дело,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Sing,nsubj,5
3,3,не,не,PART,None,advmod,5
4,4,шутка,шутка,NOUN,Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing,root,0
5,5,",",",",PUNCT,None,punct,13
6,6,ведь,ведь,SCONJ,None,mark,13
7,7,наша,наш,DET,Case=Nom|Gender=Fem|Number=Sing|Poss=Yes|PronT...,det,9
8,8,малютка,малютка,NOUN,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,nsubj,13
9,9,",",",",PUNCT,None,punct,11


In [11]:
aliсe_by_word.to_csv('alice_corpus_words.tsv', sep = '\t', encoding='utf-8',  index = False)

In [ ]:
alice_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2025/main/Practical_4/alice_corpus_words.tsv'

alice_by_word = pd.read_csv(alice_link, sep='\t')

## Работа с таблицами с помощью Pandas

С полученной таблицей можно сделать очень много всего: извлечь любые данные, аггрегировать, представить в виде, удобном для построения графиков или статистического анализа.

Мы будем это делать с помощью специально библиотеки Pandas. Это очень мощный инструмент, который имеет много встроенных операций, которые можно производить над таблицами, а также даёт возможность применять к ним произвольные функции.

В Pandas некоторые логические операторы выглядят привычным образом, а некоторые нет, например:

- == — равно
- != — не равно
- & — и
- | — или

Действуют и другие (>, < и т.д.).

Помните, что скобками удобно задать порядок операций!

In [ ]:
#Ищем по любой колонке — например, по лемме

aliсe_by_word[aliсe_by_word['lemma'] == 'день'].head(15)

,index,word,lemma,pos,feats,synt,head
74,74,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl,20
214,214,дней,день,NOUN,Animacy=Inan|Case=Gen|Gender=Masc|Number=Plur,nmod,3
1608,1608,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl,8
2986,2986,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,nsubj,5
9270,9270,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,nmod,2
10712,273,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,nsubj,10
10873,434,день,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl,21
11890,1451,день,день,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,conj,5
11892,1453,днем,день,NOUN,Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing,nmod,12
12406,1967,днем,день,NOUN,Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing,obl,8


In [37]:
#Как это работает? Квадратные скобки позволяют выбрать строки таблицы по шаблону
#А в скобки мы пишем тот самый шаблон в виде логического выражения

In [41]:
#Ищем по больше, чем одному параметру

aliсe_by_word[(aliсe_by_word['lemma'] == 'стекло') & (aliсe_by_word['pos'] == 'NOUN')]

,index,word,lemma,pos,feats,synt,head,len_word
2417,2417,стекла,стекло,NOUN,Animacy=Inan|Case=Nom|Gender=Neut|Number=Plur,nsubj:pass,28,6


In [52]:
#Ищем любой из списка элементов

aliсe_by_word[aliсe_by_word['lemma'].isin(['дом', 'гусеница'])]

,index,word,lemma,pos,feats,synt,head,len_word
704,704,дом,дом,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obj,4,3
1384,1384,дом,дом,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl,19,3
1388,1388,дом,дом,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,conj,25,3
1589,1589,домами,дом,NOUN,Animacy=Inan|Case=Ins|Gender=Masc|Number=Plur,nmod,5,6
2392,2392,дом,дом,NOUN,Animacy=Inan|Case=Nom|Gender=Masc|Number=Sing,nsubj,25,3
2397,2397,доме,дом,NOUN,Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing,nmod,2,4
2660,2660,дом,дом,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing,obl,3,3
2964,2964,домам,дом,NOUN,Animacy=Inan|Case=Dat|Gender=Masc|Number=Plur,obl,3,5
3895,3895,Гусеница,гусеница,NOUN,Animacy=Anim|Case=Nom|Gender=Fem|Number=Sing,nsubj,10,8
5325,5325,доме,дом,NOUN,Animacy=Inan|Case=Loc|Gender=Masc|Number=Sing,advcl,56,4


#### Задание 1

Найдите все пунктуационные знаки в книге

In [35]:
#Впишите сюда свой код



In [34]:
#Создаём частотный список для столбца
#.groupby() группирует таблицу по значением одного из столбцов, собирает вместе
#После этого со сгруппированными значениями можно что-то сделать: посчитать,
#сложить, найти среднее и т. д. Методы .size() и .count() — считают элементы

aliсe_by_word.groupby('lemma').size().nlargest(15)

lemma
,       827
.       420
"       417
в       296
и       188
–       144
(       126
)       126
не      103
:        88
что      87
на       85
быть     84
этот     72
с        71
dtype: int64

#### Задание 2

Посчитайте количество лемм так, чтобы среди них не учитывались пунктуационные знаки.

In [36]:
#Впишите сюда свой код



In [12]:
# Можно искать по подстроке, а не по полному совпадению

aliсe_by_word[(aliсe_by_word['lemma'] == 'день') &
              (aliсe_by_word['feats'].str.contains('Number=Plur'))]

,index,word,lemma,pos,feats,synt,head
595,595,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl,8
2194,2194,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl,7
3682,3682,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl,14
8628,8628,дни,день,NOUN,Animacy=Inan|Case=Acc|Gender=Masc|Number=Plur,obl,3


In [16]:
#Более продвинутый уровень — к какому-то из столбцов можно применить функцию
#Например, посчитать количество символов для каждого из текстов, который у нас есть


aliсe_by_word['len_word'] = aliсe_by_word['word'].map(len)

#Можем выбрать этот столбец, чтобы посмотреть на него
aliсe_by_word['len_word'].head(15)

0     2
1     1
2     4
3     2
4     5
5     1
6     4
7     4
8     7
9     1
10    6
11    1
12    8
13    6
14    1
Name: len_word, dtype: int64

In [22]:
#Численный параметр можно аггрегировать и посчитать для неё необходимую метрику
#Например, среднее:

print(aliсe_by_word.aggregate('len_word').mean())

4.222970173985087


In [25]:
#Медиану

print(aliсe_by_word.aggregate('len_word').median())

3.0


In [24]:
#Дисперсию, и так далее

print(aliсe_by_word.aggregate('len_word').var())

10.738575371870546


#### Задание 3 (со звёздочкой)

Одна из вещей, которые мы хотели уметь делать в начале курса — находить, в какие связи слово (чаще) вступает с другими словами. Мы дошли до момента, когда можем это сделать. Это можно сделать разными способами — возможно, удобнее всего будет немного изменить разметку корпуса.

In [ ]:
#Впишите сюда свой код



## Более сложная структура корпуса

Конечно, мы можем работать с корпусом, который состоит больше, чем из одной книги. Его структура может быть разной в зависимости от ваших данных.

Может быть нужно хранить мета-информацию о текстах, разметка по словам, гипотетически может быть разметка на уровне предложений. Всю эту информацию имеет смысл не смешивать, а хранить в отдельных табличках (при этом мы фактически создаём реляционную базу данных).

Чтобы понимать, как таблички соотносятся друг с другом, нужны индексы.

Создадим корпус, который содержит новости, скачанные со страницы Панорамы.

Будем работать с готовой таблицей, в которой содержится метаинформация (при небольшом количестве элементов она может быть создана вручную, при большом — автоматически), и с ещё одной таблицей, в которой будет разметка для каждого слова.

In [26]:
#Считываем из файла табличку с метаинформацией о каждом файле (включая текст новости)

panorama_corpus_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2023/main/Seminar_5/panorama_corpus.tsv'

panorama_corpus = pd.read_csv(panorama_corpus_link, sep='\t')

In [27]:
panorama_corpus.head()

,index,date,sphere,title,text,link
0,0,4-3-2023,Общество,Гражданам разрешили добывать нефть и газ на св...,Государственная дума приняла в третьем чтении ...,https://panorama.pub/news/grazdanam-razresili-...
1,1,4-3-2023,Политика,Китай подал заявку на вступление в Союзное гос...,Заведующий Канцелярией Комиссии ЦК КПК по инос...,https://panorama.pub/news/kitaj-podal-zaavku-n...
2,2,4-3-2023,Политика,Деколонизаторки из Франции потребовали вернуть...,Активисты движения «Смерть колониализму» из Па...,https://panorama.pub/news/dekolonizatorki-iz-f...
3,3,4-3-2023,Политика,Победа России: Госдума пересмотрела результаты...,Государственная дума приняла постановление «Об...,https://panorama.pub/news/gosduma-peresmotrela...
4,4,3-3-2023,Общество,Бездетных россиян будут ежегодно штрафовать,Государственная дума приняла в первом чтении з...,https://panorama.pub/news/bezdetnyh-rossian-pr...


In [28]:
#Возьмём для скорости поменьше новостей

panorama_corpus = panorama_corpus.iloc[:50]

In [29]:
#Пройдёмся по каждому слову каждой из новостей и добавим в новую табличку
cols = ['index', 'text_index', 'word', 'lemma', 'pos', 'feats',
        'synt',  'head']
words = []

for text_index, row in enumerate(panorama_corpus['text']):
  news_nlp = nlp(row)
  for word_index, word in enumerate(news_nlp.iter_words()):
    words.append([word_index, text_index, word.text, word.lemma, word.upos,
                  word.feats, word.deprel, word.head])

panorama_by_word = pd.DataFrame(words, columns = cols)

In [30]:
panorama_by_word.head()

,index,text_index,word,lemma,pos,feats,synt,head
0,0,0,Государственная,государственный,ADJ,Case=Nom|Degree=Pos|Gender=Fem|Number=Sing,amod,2
1,1,0,дума,дума,NOUN,Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing,nsubj,3
2,2,0,приняла,принять,VERB,Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Te...,root,0
3,3,0,в,в,ADP,None,case,6
4,4,0,третьем,третий,ADJ,Case=Loc|Degree=Pos|Gender=Neut|Number=Sing,amod,6


In [31]:
panorama_by_word.to_csv('panorama_corpus_words.tsv', sep = '\t', encoding='utf-8',  index = False)

In [32]:
panorama_by_word_link = 'https://raw.githubusercontent.com/alekseyst/text_analysis_2024/main/Practical_4/panorama_corpus_words.tsv'

panorama_by_word = pd.read_csv(panorama_by_word_link, sep='\t')

In [33]:
#Если нужно использовать информацию сразу из двух таблиц, их можно объединить
#Только не целиком — иначе получится слишком огромная таблица, которая потратит
#Все наши ресурсы!

#Получим таблицу, которая содержит дату создания примера для каждого слова.
#Объединять будем по индексу текста

panorama_by_word_merged = panorama_by_word.join(
    panorama_corpus.loc[:, ['index', 'date']].set_index('index'),
    on='text_index')

panorama_by_word_merged

,index,text_index,word,lemma,pos,feats,synt,head,date
0,0,0,Государственная,государственный,ADJ,Case=Nom|Degree=Pos|Gender=Fem|Number=Sing,amod,2,4-3-2023
1,1,0,дума,дума,NOUN,Animacy=Inan|Case=Nom|Gender=Fem|Number=Sing,nsubj,3,4-3-2023
2,2,0,приняла,принять,VERB,Aspect=Perf|Gender=Fem|Mood=Ind|Number=Sing|Te...,root,0,4-3-2023
3,3,0,в,в,ADP,NaN,case,6,4-3-2023
4,4,0,третьем,третий,ADJ,Case=Loc|Degree=Pos|Gender=Neut|Number=Sing,amod,6,4-3-2023
...,...,...,...,...,...,...,...,...,...
9447,145,49,организацией,организация,NOUN,Animacy=Inan|Case=Ins|Gender=Fem|Number=Sing,obl,4,24-2-2023
9448,146,49,",",",",PUNCT,NaN,punct,4,24-2-2023
9449,147,49,ещё,еще,ADV,Degree=Pos,advmod,8,24-2-2023
9450,148,49,оценивается,оценивать,VERB,Aspect=Imp|Mood=Ind|Number=Sing|Person=3|Tense...,root,0,24-2-2023
